The following codes  
https://towardsdatascience.com/selenium-tweepy-to-scrap-tweets-from-tweeter-and-analysing-sentiments-1804db3478ac
https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/  
https://medium.freecodecamp.org/how-to-make-your-own-sentiment-analyzer-using-python-and-googles-natural-language-api-9e91e1c493e  
https://www.programcreek.com/python/example/76301/tweepy.Cursor  
https://cmry.github.io/notes/twitter-python


In [1]:
# !pip3 install tweepy

In [1]:
import os,sys,inspect
import query_terms
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
import twitterkeys
import tweepy
import pandas as pd 
from tweepy import OAuthHandler

In [2]:
consumer_key = twitterkeys.CONSUMER_KEY
consumer_secret   = twitterkeys.CONSUMER_SECRET
access_token   = twitterkeys.ACCESS_TOKEN
access_token_secret   = twitterkeys.ACCESS_TOKEN_SECRET

In [3]:
angry_query_a = query_terms.ANGRY_QUERY_a
angry_query_b = query_terms.ANGRY_QUERY_b
angry_query_c = query_terms.ANGRY_QUERY_c
fear_query_a = query_terms.FEAR_QUERY_a
fear_query_b = query_terms.FEAR_QUERY_b
fear_query_c = query_terms.FEAR_QUERY_c
fear_query_d = query_terms.FEAR_QUERY_d
sad_query_a = query_terms.SAD_QUERY_a
sad_query_b = query_terms.SAD_QUERY_b
sad_query_c = query_terms.SAD_QUERY_c
sad_query_d = query_terms.SAD_QUERY_d

In [4]:
def authentication(cons_key, cons_secret, acc_token, acc_secret):
    auth = tweepy.OAuthHandler(cons_key, cons_secret)
    auth.set_access_token(acc_token, acc_secret)
    api = tweepy.API(auth)
    return api

In [5]:
from datetime import datetime, timedelta
today_datetime = datetime.today().now()
yesterday_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')

## Test scrapping mechanism

In [16]:
def search_tweets_no_date(keyword, total_tweets):
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    api = authentication(consumer_key,consumer_secret,access_token,access_token_secret)
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since=yesterday_date, 
                                  result_type='recent', 
                                  tweet_mode='extended',
                                  lang='en').items(total_tweets)
    
    # Add tweets in this format
    listOfTweets = []
    
    print(yesterday_date, yesterday_datetime)
    for tweet in search_result:
        # Add tweets in this format
        if tweet.user.followers_count < 100:
            continue
        else:
            dict_ = {'Screen Name': tweet.user.screen_name,
                    'User Name': tweet.user.name,
                    'User Location': tweet.user.location,       
                    'User Likes':tweet.user.favourites_count,
                    'User Followers':tweet.user.followers_count,
                    'User Following':tweet.user.friends_count,
                    'User Total Tweets':tweet.user.statuses_count, 
                    'User Created Time': tweet.user.created_at,
                    'User Description': tweet.user.description,

                    'Tweet Text': tweet.full_text,                                  
                    'Tweet Created Time': tweet.created_at,
                    'Tweet Likes': tweet.favorite_count,
                    'Tweet Retweet Count':tweet.retweet_count,                 
                    'Tweet Coordinates': tweet.coordinates,                
                    'Tweet As Retweet': tweet.retweeted,
                    'Tweet Platform Type': tweet.source,
                    'Tweet Language': tweet.lang,


                    'Tweet Favorited': tweet.favorited,
                    'Tweet In reply to': tweet.in_reply_to_status_id_str,
                    'Tweet id': tweet.id_str,
                    'Tweet URL': 'https://twitter.com/i/web/status/'+ tweet.id_str
                    }
            listOfTweets.append(dict_)   
    return pd.DataFrame(listOfTweets)

In [17]:
# today_datetime = datetime.today().now()
# yesterday_datetime = today_datetime - timedelta(days=1)
# today_date = today_datetime.strftime('%Y-%m-%d')
# yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
# angry_b = search_tweets_no_date(angry_query_a,100)
# angry_b

### Test saving and loading from csv and pandas

In [50]:
# today_datetime = datetime.today().now()
# yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
# filename = '/data/SuperMod/TwitterData/anger' + yesterday_date + '.csv'

In [51]:

# angry_b.to_csv(path_or_buf =filename , index=False)

In [52]:
# pd = pd.read_csv(filename)
# pd

,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,mrithejester,False,NaN,2019-05-27 08:24:09,False,NaN,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132925446134...,1132925446134358016,2018-01-13 12:22:00,I exist. No religion. No DMs. 🚫,1993,441,25413,K2-229 b,Paul,24730
1,satya_AmitSingh,False,NaN,2019-05-27 08:24:07,False,1.132916e+18,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132925437166...,1132925437166800896,2011-12-22 04:27:02,"Nationalist 🇮🇳Proud Hindu🧡& Entrepreneur,CEO w...",259,168,10482,"New Delhi, Delhi",Satyajit Amit Singh,4019
2,DumbxssHooker,False,NaN,2019-05-27 08:24:05,False,NaN,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1132925429814...,1132925429814366208,2019-02-16 23:27:37,⠀⠀⠀❪ ·𖥸· ❫ ｜ ➻ 〝 ᴀʀᴇ ʏᴏᴜ ᴍᴀᴅ ᴏʀ ᴀʀᴇ ʏᴏᴜ ꜱᴛᴀʀɪɴ...,195,142,293,#𝔾𝕚𝕟𝕘𝕖𝕣ℙ𝕒𝕤𝕥𝕪 | #Fᴀᴋᴇ,ㅤ ㅤ𝐑. 𝐋𝐞𝐬𝐭𝐫𝐚𝐧𝐠𝐞,146


In [15]:
# api.rate_limit_status()['resources']

## put everything in functions

In [35]:
# rewrite this function again for scrapping

def search_tweets(keyword, total_tweets, last_time, cur_time):

    api = authentication(consumer_key,consumer_secret,access_token,access_token_secret)
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since = last_time, 
#                                   until= cur_time,
                                  result_type='recent', 
                                  tweet_mode='extended',
                                  lang='en').items(total_tweets)
    
    # Add tweets in this format
    listOfTweets = []
    
    for tweet in search_result:
        # Add tweets in this format
        if tweet.user.followers_count < 100:
            continue
        else:
            dict_ = {'Screen Name': tweet.user.screen_name,
                    'User Name': tweet.user.name,
                    'User Location': tweet.user.location,       
                    'User Likes':tweet.user.favourites_count,
                    'User Followers':tweet.user.followers_count,
                    'User Following':tweet.user.friends_count,
                    'User Total Tweets':tweet.user.statuses_count, 
                    'User Created Time': tweet.user.created_at,
                    'User Description': tweet.user.description,

                    'Tweet Text': tweet.full_text,                                  
                    'Tweet Created Time': tweet.created_at,
                    'Tweet Likes': tweet.favorite_count,
                    'Tweet Retweet Count':tweet.retweet_count,                 
                    'Tweet Coordinates': tweet.coordinates,                
                    'Tweet As Retweet': tweet.retweeted,
                    'Tweet Platform Type': tweet.source,
                    'Tweet Language': tweet.lang,


                    'Tweet Favorited': tweet.favorited,
                    'Tweet In reply to': tweet.in_reply_to_status_id_str,
                    'Tweet id': tweet.id_str,
                    'Tweet URL': 'https://twitter.com/i/web/status/'+ tweet.id_str
                    }
            listOfTweets.append(dict_)   
    return pd.DataFrame(listOfTweets)

def scrape_by_date(list_of_sizes, days_ago, skip_fear):
    """
    This function scrapes Twitter for a certain number of messages for different query words
    and scrapes for a specific day from today
    list_of_sizes: list of number of tweets to scrape by keyword lists. 
                   Not all queries return enough words, if don't specify will run into issues
    days_ago: an integer, specify how many days ago do we want to scrape, 
             today is 1/10/19, if I put 2, then I will be scrapping from 1/7/19 to 1/8/19
    skip_fear: do not scrape for fear if True, on certain days there are simply not enough to scrape
    """
    
    today_datetime = datetime.today().now()
    
    end_datetime = today_datetime - timedelta(days = days_ago)
    start_datetime = end_datetime - timedelta(days = 1)
    
    start_date = start_datetime.strftime('%Y-%m-%d')
    end_date = end_datetime.strftime('%Y-%m-%d')
    
    print("scrapping from date", start_date, "to date", end_date)
    
    
    angry_a = search_tweets(angry_query_a, list_of_sizes[0], start_date, end_date)
    angry_b = search_tweets(angry_query_b, list_of_sizes[1], start_date, end_date)
    angry_c = search_tweets(angry_query_c, list_of_sizes[2], start_date, end_date)
    
    sad_a = search_tweets(sad_query_a, list_of_sizes[7], start_date, end_date)
    sad_b = search_tweets(sad_query_b, list_of_sizes[8], start_date, end_date)
    sad_c = search_tweets(sad_query_c, list_of_sizes[9], start_date, end_date)       
    sad_d = search_tweets(sad_query_d, list_of_sizes[10], start_date, end_date)      

    angry = angry_a.append([angry_b,angry_c])
    sad = sad_a.append([sad_b, sad_c, sad_d])   
            
    base_path = '/data/SuperMod/TwitterData/'

    filenameanger = base_path + 'anger/' + start_date + '.csv'    
    filenamesad = base_path + 'sad/' + start_date + '.csv'  
    
    if len(angry) != 0:
        angry.to_csv(path_or_buf =filenameanger , index=False)
    if len(sad) != 0:
        sad.to_csv(path_or_buf =filenamesad , index=False)

    if not skip_fear:
        
        fear_a = search_tweets(fear_query_a, list_of_sizes[3], start_date, end_date)
        fear_b = search_tweets(fear_query_b, list_of_sizes[4], start_date, end_date)
        fear_c = search_tweets(fear_query_c, list_of_sizes[5], start_date, end_date) 
        fear_d = search_tweets(fear_query_d, list_of_sizes[6], start_date, end_date)   
    
        fear = fear_a.append([fear_b, fear_c, fear_d])
        
        print("angry:" ,len(angry), "fear:", len(fear), "sad:", len(sad))
    
        filenamefear = base_path + 'fear/' + start_date + '.csv'   
        
        if len(fear) != 0: 
            fear.to_csv(path_or_buf =filenamefear , index=False)

    else: 
        print("angry:" ,len(angry),  "sad:", len(sad))   


## On some days there may not be that many tweets with emotional indicators
have to 

In [36]:
##
sizes_lar = [100, 100, 100, 5, 10, 10, 10, 10, 100, 100, 100]
sizes_med = [10, 50, 50, 5, 10, 10, 10, 10, 50, 50, 50]
sizes_sma = [10, 10, 10, 5, 10, 10, 10, 10, 10, 10, 10]



In [39]:
days = 7

In [40]:


try:
    print("scraping large size with fear")
    scrape_by_date(sizes_lar, days, False)
except:
    try:
        print("can't scrape large size with fear")
        print("scraping medium size with fear")        
        scrape_by_date(sizes_med, days, False)
    except:
        print("can't scrape medium size with fear")
        print("scraping large size without fear")
        try: 
            scrape_by_date(sizes_lar, days, True)
        except:
            print("can't scrape large size without fear")
            print("scraping medium size without fear")
            try:
                scrape_by_date(sizes_med, days, True)
            except:
                print("can't scrape medium size without fear")
                print("scraping small size without fear")       
                scrape_by_date(sizes_sma, days, True)



scraping large size with fear
scrapping from date 2019-05-20 to date 2019-05-21
angry: 211 fear: 22 sad: 213


In [ ]:
today_datetime = datetime.today().now()
today_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
print(today_date)

In [52]:
today_datetime = datetime.today().now()- timedelta(days=3)
yesterday_datetime = today_datetime - timedelta(days=1)
today_date = today_datetime.strftime('%Y-%m-%d')
yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
print("scrapping from date", yesterday_date, "to date", today_date)
angry_b = search_tweets(angry_query_a,10,yesterday_date,today_date)
angry_b

scrapping from date 2019-05-24 to date 2019-05-25


,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,Kendalsnowflake,False,None,2019-05-24 23:59:59,False,1132071306592174085,en,3,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073793281...,1132073793281642497,2010-05-22 18:18:37,Director. Cinematographer. Seahawks fan. Socia...,181,349,8513,"Los Angeles, CA",Kendall Christianson,8864
1,GButtafly,False,None,2019-05-24 23:59:58,False,None,en,4,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073788160...,1132073788160565254,2018-02-16 05:42:50,Findom Goddess. ISO pay 🐷 💰 💵 I’ll take all of...,165,71,8,,🦋GoddessButtafly 🦋,248
2,tgs102002,False,None,2019-05-24 23:59:58,False,1132062674257211392,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073787170...,1132073787170729986,2011-03-27 11:27:51,"yup, these are the things I think about.",114,259,2372,Chicago,t.g.s,2659
3,tiffanytrent,False,None,2019-05-24 23:59:57,False,1132072940059734016,en,1,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073783358...,1132073783358119936,2009-03-09 22:25:22,Author of science fiction and fantasy for adul...,3268,1922,28179,"Here, there, and everywhere",Tiffany Trent,43057
4,ashleyecrosby,False,None,2019-05-24 23:59:57,False,None,en,3,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073781646...,1132073781646766081,2009-11-19 18:21:05,Strength & Nutrition Coach | Always at a baseb...,976,658,7905,"Jupiter, FL","Ashley Wittgren, MS CSCS",23337
5,Reematefarag,False,None,2019-05-24 23:59:54,False,None,en,1,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1132073772654...,1132073772654239745,2013-02-06 13:21:50,مش هعمل فولو باك لكل الناس لسبب ما، هندسة .. ا...,2319,288,72275,"Misr Al-Gadeeda,Cairo,Egypt","As a matter of fact, I",61688
6,saba_dee,False,None,2019-05-24 23:59:51,False,1132073389550657538,en,1,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073759530...,1132073759530278913,2014-02-26 02:31:05,Eagle 🦅|| Sociologist || Introvert || God love...,386,373,6169,"Ibadan, Nigeria",King Saba,3402
7,YolandHowden,False,None,2019-05-24 23:59:46,False,1131851246317518848,en,1,Twitter for Android,0,...,https://twitter.com/i/web/status/1132073736667...,1132073736667111424,2009-10-26 23:36:14,If you're happy with your floorboards no-one c...,203,275,1695,Staffordshire,Yoland Howden,1527


### read in results to check

In [23]:
filename = '/data/SuperMod/TwitterData/anger/2019-04-27.csv'

In [26]:
test_load = pd.read_csv(filename)
test_load.head()

,Screen Name,Tweet As Retweet,Tweet Coordinates,Tweet Created Time,Tweet Favorited,Tweet In reply to,Tweet Language,Tweet Likes,Tweet Platform Type,Tweet Retweet Count,...,Tweet URL,Tweet id,User Created Time,User Description,User Followers,User Following,User Likes,User Location,User Name,User Total Tweets
0,fallenfists,False,NaN,2019-05-28 02:30:37,False,1.133197e+18,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133198865367...,1133198865367293952,2017-03-13 04:15:21,* you are filled with BRAVERY.,166,171,11,NaN,magnus.,152
1,VaccinateLux,False,NaN,2019-05-28 02:30:37,False,NaN,en,0,Twitter for Android,0,...,https://twitter.com/i/web/status/1133198862011...,1133198862011682826,2017-03-23 22:18:36,Hulk Hatter 🎩😈🙊🙅💃\n\nFollow my backup account ...,555,366,13315,United States,La Diablita 🎩😈,4562
2,sdscannevin,False,NaN,2019-05-28 02:30:36,False,NaN,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133198858954...,1133198858954117122,2009-03-18 20:36:59,NaN,106,770,3651,"Boston, MA",Sean Scannevin,5167
3,Ryanexpress77,False,NaN,2019-05-28 02:30:36,False,1.133197e+18,en,0,Twitter Web Client,0,...,https://twitter.com/i/web/status/1133198858287...,1133198858287288321,2009-07-14 17:02:57,"USC Football, Angels Baseball, John Wayne, and...",2177,1665,63,Anaheim/Dallas/Maui,Thomas Marchetti,91278
4,Frank_Frangie,False,NaN,2019-05-28 02:30:35,False,1.133198e+18,en,0,Twitter for iPhone,0,...,https://twitter.com/i/web/status/1133198856081...,1133198856081018880,2012-01-11 03:39:50,"Broadcaster for more years than I can count, A...",15165,220,8158,NaN,Frank Frangie,22613


In [29]:
for i in test_load['Tweet Text']:
    print(i)

@kindnessremains ||: I like animosity...
Seriously? 🤦🤦🤦. Always bragging about her degree and books that getting knocked up at 16 made possible. As if she actually had to work hard. And nope @KailLowry you haven't grown up at all. Still an immature, miserable, bitter baby mama. Oh and your podcast sucks! 😂🤣 https://t.co/R4AEhSpNwi
Brian Windhorst refusing to make predictions in ESPN features where he’s asked to make one never ceases to infuriate me. Dude is the luckiest loser around. Anyway, glad the #NBAFinals are here.
@parrhizzia @AJDelgado13 @soledadobrien The Trump DOJ should use the same words the Obama DOJ used in backing up Holder. https://t.co/9p0KcOIj0m
@wvnottingham Ha. Ok you’re probably right. But indulge my angry fan guy moment will you? It’s cathartic.😁
@Menace_Bunny @khtagh @iwantmynbn By pitting one group of workers against the other, placing value on one lot of workers, to not be angry that there is no 100% certainty that ppl won’t be living in poverty if the Greens p